In [ ]:
import pandas as pd
import numba as nb
import numpy as np
import re
import ast

Read the csv created by destination_transfer.ipynb


In [ ]:
df = pd.read_csv('Feb4AMDF.csv')
df.head()

,Unnamed: 0,cardID,date,DPAYPOINT,DENOMINAPARADA,LATITUD,LONGITUD,type,IDLINEA,IDSTOP,transfer,destinaiton
0,0,0000C2E53356EC1888BCD5FC71669150,2020-02-01 11:24:44,03_L138_P366,NaN,40.37792,-3.77278,1,138,2410,False,-2.0
1,1,0000C2E53356EC1888BCD5FC71669150,2020-02-01 12:40:20,03_L138_P578,NaN,40.38360,-3.76524,1,138,1231,False,-2.0
2,2,0000C2E53356EC1888BCD5FC71669150,2020-02-01 12:49:58,03_L39_P366,NaN,40.37792,-3.77278,1,39,2410,False,-1.0
3,3,0000F43B3D2019E8F90D4B612E834D92,2020-02-01 11:30:04,2C_L551_P1073,NaN,40.38292,-4.20947,1,551,13041,True,4754.0
4,4,0000F43B3D2019E8F90D4B612E834D92,2020-02-01 12:18:00,02_L5_P29,CAMPAMENTO,40.39481,-3.76813,0,NaN,49,False,-2.0


Number of the transfer detected by the algorithm.

In [ ]:
df['transfer'].value_counts()

False    315491
True      83774
Name: transfer, dtype: int64

Number of the unestimated destination by their categories;


*   -1: The final destination could not be estimated.
*   -2: The intermediate transfers could not estimated.



In [ ]:
dest1 = ((df['destinaiton'] == -1) & (df['transfer'] == False)).sum()
dest2 = ((df['destinaiton'] == -2) & (df['transfer'] == False)).sum()

print(dest1)
print(dest2)


61313
163289


List created to obtain the number of transfers that are the middle elements of the transfer chains which contains more than 2 consecutive transfers. Later, the intermediate transfers are dropped

In [ ]:
ListTrue =[]
for row in range(len(df)-1):
  if df.iloc[row, 10] == True & df.iloc[row+1, 10] == True:
   ListTrue.append(row+1)
print(len(ListTrue))
df.drop(ListTrue, axis=0, inplace=True)

20818


Destination column is shifted one up and save to the new column called destShifted.

In [ ]:
df['destShifted'] = df['destinaiton'].shift(-1, axis = 0)
df.head()

,Unnamed: 0,cardID,date,DPAYPOINT,DENOMINAPARADA,LATITUD,LONGITUD,type,IDLINEA,IDSTOP,transfer,destinaiton,destShifted
0,0,0000C2E53356EC1888BCD5FC71669150,2020-02-01 11:24:44,03_L138_P366,NaN,40.37792,-3.77278,1,138,2410,False,-2.0,-2.0
1,1,0000C2E53356EC1888BCD5FC71669150,2020-02-01 12:40:20,03_L138_P578,NaN,40.38360,-3.76524,1,138,1231,False,-2.0,-1.0
2,2,0000C2E53356EC1888BCD5FC71669150,2020-02-01 12:49:58,03_L39_P366,NaN,40.37792,-3.77278,1,39,2410,False,-1.0,4754.0
3,3,0000F43B3D2019E8F90D4B612E834D92,2020-02-01 11:30:04,2C_L551_P1073,NaN,40.38292,-4.20947,1,551,13041,True,4754.0,-2.0
4,4,0000F43B3D2019E8F90D4B612E834D92,2020-02-01 12:18:00,02_L5_P29,CAMPAMENTO,40.39481,-3.76813,0,NaN,49,False,-2.0,282.0


The destination value of the row is changed as;



1.   destShifted, if the row has transfer True.
2.   destination, if the row has transfer False.

Because, if there is a transfe we want to combine the origin of the first trip and destination of the last trip of a transfer chain into one record.



In [ ]:
df['destinaiton'] = df['destShifted'] * (df.transfer == True) + df['destinaiton'] * (df.transfer == False)
df.head()

,Unnamed: 0,cardID,date,DPAYPOINT,DENOMINAPARADA,LATITUD,LONGITUD,type,IDLINEA,IDSTOP,transfer,destinaiton,destShifted
0,0,0000C2E53356EC1888BCD5FC71669150,2020-02-01 11:24:44,03_L138_P366,NaN,40.37792,-3.77278,1,138,2410,False,-2.0,-2.0
1,1,0000C2E53356EC1888BCD5FC71669150,2020-02-01 12:40:20,03_L138_P578,NaN,40.38360,-3.76524,1,138,1231,False,-2.0,-1.0
2,2,0000C2E53356EC1888BCD5FC71669150,2020-02-01 12:49:58,03_L39_P366,NaN,40.37792,-3.77278,1,39,2410,False,-1.0,4754.0
3,3,0000F43B3D2019E8F90D4B612E834D92,2020-02-01 11:30:04,2C_L551_P1073,NaN,40.38292,-4.20947,1,551,13041,True,-2.0,-2.0
4,4,0000F43B3D2019E8F90D4B612E834D92,2020-02-01 12:18:00,02_L5_P29,CAMPAMENTO,40.39481,-3.76813,0,NaN,49,False,-2.0,282.0


The records without the destination value (which means their destinations could not be estimated therefore the default values, -1, and -2 are assigned to the destination column) are removed.

In [ ]:
df = df[df.destinaiton > 0]
df.head()

,Unnamed: 0,cardID,date,DPAYPOINT,DENOMINAPARADA,LATITUD,LONGITUD,type,IDLINEA,IDSTOP,transfer,destinaiton,destShifted
5,5,0000F43B3D2019E8F90D4B612E834D92,2020-02-01 14:58:02,02_L5_P25,CARABANCHEL,40.38783,-3.74487,0,NaN,54,True,13041.0,13041.0
6,6,0000F43B3D2019E8F90D4B612E834D92,2020-02-01 15:38:28,2C_L551_P1049,NaN,40.39657,-3.76835,1,551,4754,False,13041.0,-2.0
10,10,00020E8A68C55F688FDA981B76AFFECC,2020-02-01 11:51:36,02_L2_P1,VENTAS,40.43088,-3.66361,0,NaN,273,False,222.0,-2.0
14,14,0002CF6E367108EAA638D4538894DC42,2020-02-01 11:56:08,03_L68_P2213,NaN,40.41140,-3.67648,1,68,3385,False,2558.0,-2.0
19,19,0002CF6E367108EAA638D4538894DC42,2020-02-01 18:22:28,03_L68_P421,NaN,40.44611,-3.69040,1,68,1940,False,3385.0,-2.0


The trip legs of the transfer chains are removed except the first one.

In [ ]:
df['transferShifted'] = df['transfer'].shift(1, axis = 0)
df = df[df['transferShifted'] == False]
df.head()

,Unnamed: 0,cardID,date,DPAYPOINT,DENOMINAPARADA,LATITUD,LONGITUD,type,IDLINEA,IDSTOP,transfer,destinaiton,destShifted,transferShifted
10,10,00020E8A68C55F688FDA981B76AFFECC,2020-02-01 11:51:36,02_L2_P1,VENTAS,40.43088,-3.66361,0,NaN,273,False,222.0,-2.0,False
14,14,0002CF6E367108EAA638D4538894DC42,2020-02-01 11:56:08,03_L68_P2213,NaN,40.41140,-3.67648,1,68,3385,False,2558.0,-2.0,False
19,19,0002CF6E367108EAA638D4538894DC42,2020-02-01 18:22:28,03_L68_P421,NaN,40.44611,-3.69040,1,68,1940,False,3385.0,-2.0,False
21,21,000342C0DCC05B5206098B5DC3CDDAEE,2020-02-01 17:57:20,02_L3_P7,CALLAO,40.42014,-3.70566,0,NaN,47,False,261.0,-2.0,False
25,25,0003F3A147FA408E1E0837E398603D76,2020-02-01 13:27:50,03_L28_P962,NaN,40.42574,-3.65163,1,28,3164,False,3027.0,-2.0,False


The unnecessary columns are removed.

In [ ]:
OD_Df = df.drop(columns=[ 'cardID', 'transfer', 'destShifted', 'transferShifted'])
OD_Df['destinaiton'] = OD_Df['destinaiton'].astype(int)
OD_Df

,Unnamed: 0,date,DPAYPOINT,DENOMINAPARADA,LATITUD,LONGITUD,type,IDLINEA,IDSTOP,destinaiton
10,10,2020-02-01 11:51:36,02_L2_P1,VENTAS,40.43088,-3.66361,0,NaN,273,222
14,14,2020-02-01 11:56:08,03_L68_P2213,NaN,40.41140,-3.67648,1,68,3385,2558
19,19,2020-02-01 18:22:28,03_L68_P421,NaN,40.44611,-3.69040,1,68,1940,3385
21,21,2020-02-01 17:57:20,02_L3_P7,CALLAO,40.42014,-3.70566,0,NaN,47,261
25,25,2020-02-01 13:27:50,03_L28_P962,NaN,40.42574,-3.65163,1,28,3164,3027
...,...,...,...,...,...,...,...,...,...,...
399253,399253,2020-02-01 13:59:18,33_L484_P320,NaN,40.33760,-3.75434,1,484,7124,10286
399256,399256,2020-02-01 20:45:40,33_L484_P243,NaN,40.32494,-3.76653,1,484,5679,5316
399259,399259,2020-02-01 12:03:56,2F_L520_P90,NaN,40.34621,-3.82447,1,520,11789,8353
399260,399260,2020-02-01 13:00:22,26_L621_P43,NaN,40.49179,-3.87315,1,621,7647,6274


In [ ]:
stationInfoDf = pd.read_csv('stationInfo.csv', usecols=['CODIGOMUNICIPIO','DPAYPOINT'])
stopInfoDf = pd.read_csv('stopInfo.csv', usecols=['CODIGOMUNICIPIO','DPAYPOINT'])

stopsMunicipio = pd.concat([stationInfoDf, stopInfoDf]).drop_duplicates('DPAYPOINT')
stopsMunicipio

,CODIGOMUNICIPIO,DPAYPOINT
0,28079,02_L11_P2
1,28079,02_L5_P19
2,28079,02_L5_P52
3,28007,02_L12_P3
4,28079,02_L4_P13
...,...,...
36386,28058,99_L471_P1950
36387,28115,3F_L10_P1078
36388,28079,33_L492_P124
36389,28176,26_L627_P58


In the following 3 code block, the district code of origin and destination locations are added to the data frame.

In [ ]:
OD_Df_municipio = pd.merge(OD_Df, stopsMunicipio, on=['DPAYPOINT'], how='left')

,Unnamed: 0,date,DPAYPOINT,DENOMINAPARADA,LATITUD,LONGITUD,type,IDLINEA,IDSTOP,destinaiton,CODIGOMUNICIPIO
0,10,2020-02-01 11:51:36,02_L2_P1,VENTAS,40.43088,-3.66361,0,NaN,273,222,28079
1,14,2020-02-01 11:56:08,03_L68_P2213,NaN,40.41140,-3.67648,1,68,3385,2558,28079
2,19,2020-02-01 18:22:28,03_L68_P421,NaN,40.44611,-3.69040,1,68,1940,3385,28079
3,21,2020-02-01 17:57:20,02_L3_P7,CALLAO,40.42014,-3.70566,0,NaN,47,261,28079
4,25,2020-02-01 13:27:50,03_L28_P962,NaN,40.42574,-3.65163,1,28,3164,3027,28079
...,...,...,...,...,...,...,...,...,...,...,...
90874,399253,2020-02-01 13:59:18,33_L484_P320,NaN,40.33760,-3.75434,1,484,7124,10286,28074
90875,399256,2020-02-01 20:45:40,33_L484_P243,NaN,40.32494,-3.76653,1,484,5679,5316,28074
90876,399259,2020-02-01 12:03:56,2F_L520_P90,NaN,40.34621,-3.82447,1,520,11789,8353,28007
90877,399260,2020-02-01 13:00:22,26_L621_P43,NaN,40.49179,-3.87315,1,621,7647,6274,28127


In [ ]:
OD_Df_municipio = pd.merge(OD_Df_municipio, allUniqueStopDf[['DPAYPOINT', 'IDSTOP']], left_on=['destinaiton'], right_on=['IDSTOP'], how='left')
OD_Df_municipio = pd.merge(OD_Df_municipio, stopsMunicipio, left_on=['DPAYPOINT_y'], right_on=['DPAYPOINT'], how='left')

In [ ]:
OD_Df_municipio['ODpair'] = OD_Df_municipio['CODIGOMUNICIPIO_x'].astype(str)+'-'+OD_Df_municipio['CODIGOMUNICIPIO_y'].astype(str)
OD_Df_municipio = OD_Df_municipio.drop(columns=[ 'DPAYPOINT', 'DPAYPOINT_y', 'DPAYPOINT_x', 'IDLINEA', 'DENOMINAPARADA', 'IDSTOP_y' ])
OD_Df_municipio

,Unnamed: 0,date,LATITUD,LONGITUD,type,IDSTOP_x,destinaiton,CODIGOMUNICIPIO_x,CODIGOMUNICIPIO_y,ODpair
0,10,2020-02-01 11:51:36,40.43088,-3.66361,0,273,222,28079,28079,28079-28079
1,14,2020-02-01 11:56:08,40.41140,-3.67648,1,3385,2558,28079,28079,28079-28079
2,19,2020-02-01 18:22:28,40.44611,-3.69040,1,1940,3385,28079,28079,28079-28079
3,21,2020-02-01 17:57:20,40.42014,-3.70566,0,47,261,28079,28079,28079-28079
4,25,2020-02-01 13:27:50,40.42574,-3.65163,1,3164,3027,28079,28079,28079-28079
...,...,...,...,...,...,...,...,...,...,...
90874,399253,2020-02-01 13:59:18,40.33760,-3.75434,1,7124,10286,28074,28074,28074-28074
90875,399256,2020-02-01 20:45:40,40.32494,-3.76653,1,5679,5316,28074,28074,28074-28074
90876,399259,2020-02-01 12:03:56,40.34621,-3.82447,1,11789,8353,28007,28007,28007-28007
90877,399260,2020-02-01 13:00:22,40.49179,-3.87315,1,7647,6274,28127,28079,28127-28079


Counting the district (municipio) pairs for whole day. 

In [ ]:
ODpair_count = pd.DataFrame(OD_Df_municipio['ODpair'].value_counts(sort=True))
ODpair_count_reset = ODpair_count.reset_index()
ODpair_count_reset.columns = ['municipio', 'counts']
ODpair_count_reset

,municipio,counts
0,28079-28079,70878
1,28074-28074,1278
2,28092-28092,1124
3,28005-28005,991
4,28065-28065,906
...,...,...
783,28066-28140,1
784,28065-45161,1
785,28031-28037,1
786,28074-28113,1


In [ ]:
Od_madrid = ODpair_count_reset[ODpair_count_reset['municipio'].str.startswith(("28079"))]
starts = Od_madrid['counts'].sum()

Od_madrid = ODpair_count_reset[ODpair_count_reset['municipio'].str.endswith(("28079"))]
ends = Od_madrid['counts'].sum()

print('trips starts from Madrid center:', starts)
print('trips arrives to Madrid center:', ends)

trips starts from Madrid center: 76023
trips arrives to Madrid center: 75301


### Dividing OD Matrix into 3 periods of the day for analysis



*   Morning: 04.00-12.00
*   Afternoon: 12.00-20.00
*   Night: 20.00-04.00



Creating data fram for morning.

In [ ]:
OD_Df_morning = OD_Df_municipio[OD_Df_municipio['date'] < '2020-02-01 12:00:00']
OD_Df_morning.head()

,Unnamed: 0,date,LATITUD,LONGITUD,type,IDSTOP_x,destinaiton,CODIGOMUNICIPIO_x,CODIGOMUNICIPIO_y,ODpair
0,10,2020-02-01 11:51:36,40.43088,-3.66361,0,273,222,28079,28079,28079-28079
1,14,2020-02-01 11:56:08,40.41140,-3.67648,1,3385,2558,28079,28079,28079-28079
15,79,2020-02-01 09:05:16,40.30228,-3.73832,1,9227,209,28065,28079,28065-28079
17,89,2020-02-01 10:05:32,40.39845,-3.67005,1,7208,3175,28079,28079,28079-28079
23,107,2020-02-01 11:46:46,40.39833,-3.76261,1,3919,2374,28079,28079,28079-28079


Creating data fram for afternoon.

In [ ]:
OD_Df_afternoon = OD_Df_municipio[(OD_Df_municipio['date'] > '2020-02-01 12:00:00') & (OD_Df_municipio['date'] < '2020-02-01 20:00:00')]
OD_Df_afternoon.head()

,Unnamed: 0,date,LATITUD,LONGITUD,type,IDSTOP_x,destinaiton,CODIGOMUNICIPIO_x,CODIGOMUNICIPIO_y,ODpair
2,19,2020-02-01 18:22:28,40.44611,-3.69040,1,1940,3385,28079,28079,28079-28079
3,21,2020-02-01 17:57:20,40.42014,-3.70566,0,47,261,28079,28079,28079-28079
4,25,2020-02-01 13:27:50,40.42574,-3.65163,1,3164,3027,28079,28079,28079-28079
5,37,2020-02-01 19:11:20,40.40513,-3.70262,0,309,290,28079,28079,28079-28079
7,40,2020-02-01 13:21:52,40.41503,-3.66951,0,237,238,28079,28079,28079-28079


Creating data fram for night.

In [ ]:
OD_Df_night = OD_Df_municipio[(OD_Df_municipio['date'] > '2020-02-01 20:00:00') & (OD_Df_municipio['date'] < '2020-02-02 04:00:00')]
OD_Df_night.head()

,Unnamed: 0,date,LATITUD,LONGITUD,type,IDSTOP_x,destinaiton,CODIGOMUNICIPIO_x,CODIGOMUNICIPIO_y,ODpair
6,39,2020-02-01 21:44:46,40.43851,-3.60848,1,2664,649,28079,28079,28079-28079
9,57,2020-02-01 21:07:50,40.47689,-3.70316,0,39,98,28079,28079,28079-28079
11,67,2020-02-01 20:07:48,40.41438,-3.70362,1,468,793,28079,28079,28079-28079
13,76,2020-02-01 21:32:48,40.39219,-3.74175,1,4081,622,28079,28079,28079-28079
27,125,2020-02-01 20:55:24,40.43672,-3.63739,1,10614,10581,28079,28049,28079-28049


Counting the district (municipio) pairs for night period. 

In [ ]:
ODpair_count = pd.DataFrame(OD_Df_night['ODpair'].value_counts(sort=True))
ODpair_count_reset = ODpair_count.reset_index()
ODpair_count_reset.columns = ['municipio', 'counts']
ODpair_count_reset

,municipio,counts
0,28079-28079,15411
1,28079-28092,415
2,28079-28007,376
3,28005-28005,305
4,28074-28074,270
...,...,...
306,28022-28007,1
307,28133-28079,1
308,28079-28072,1
309,28079-28028,1


Counting the number of trips from and to Madrid center.



In [ ]:
Od_madrid = ODpair_count_reset[ODpair_count_reset['municipio'].str.startswith(("28079"))]
starts = Od_madrid['counts'].sum()

Od_madrid = ODpair_count_reset[ODpair_count_reset['municipio'].str.endswith(("28079"))]
ends = Od_madrid['counts'].sum()

print('trips starts from Madrid center:', starts)
print('trips arrives to Madrid center:', ends)

trips starts from Madrid center: 17384
trips arrives to Madrid center: 15948
